In [1]:
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid")
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr
import datetime
import yfinance as yf
yf.pdr_override()

## Funções Importantes

In [2]:
#ADF test
# MQO para encontrar o cointegrated coefficient e criando a series do spread
def OLS(data_ticker1, data_ticker2):
    spread = sm.OLS(data_ticker1,data_ticker2)
    spread = spread.fit()
    return data_ticker1 + (data_ticker2 * -spread.params[0]), spread.params[0]

def ADF(spread):
    return statsmodels.tsa.stattools.adfuller(spread)# H0: Raiz unitária.

def ADF_test(data_ticker1, data_ticker2):
    ols = OLS(data_ticker1, data_ticker2)
    spread = ols[0]
    gamma = ols[1]
    return ADF(spread),gamma

In [3]:
def find_cointegrated_pairs_mod(data):
    n = data.shape[1]
    pvalue_matrix = np.ones((n, n))
    gammas_matrix= np.ones((n, n))
    keys = data.keys()
    for i in range(n):
        for j in range(i+1, n):
            S1 = keys[i]
            S2 = keys[j]
            result = ADF_test(data[S1], data[S2])
            gammas_matrix[i, j] =result[1] # gamma
            pvalue = result[0][1] # pvalue
            pvalue_matrix[i, j] = pvalue
    return pvalue_matrix, gammas_matrix

In [4]:
# Top n pares com o menor P-Value

#data = dados com o log preço dos ativos
#pvalue_matrix = matriz de pvalores
#gamma = vetor com os gammas dos ativos
#alpha = nivel de significancia para o teste ADF
#n = top n ativos com o menor pvalue

def top_coint_pairs(data,pvalue_matrix,gamma, alpha = 0.05, n = 5): 

    pvalues_f = pvalues[np.where(pvalues < alpha)] # pvalores menores que 0.05
    stock_a = data.columns[list(np.where(pvalues < alpha))[0]] # relacionando o pvalor com a ação A
    stock_b = data.columns[list(np.where(pvalues < alpha))[1]] # relacionando o pvalor com a ação B
    gammas_f = gammas[np.where(pvalues < alpha)] # relacionando o pvalor com o gamma
    N = len(list(np.where(pvalues < alpha)[0])) # quantidade de pares cointegrados

    d = []
    for i in range(N):
      d.append(
            {
                'Stock A': stock_a[i],
                'Stock B': stock_b[i],
                'P-Values': pvalues_f[i],
                'Gamma': gammas_f[i]
            }
        )

    #pegando top 5 pares
    return pd.DataFrame(d).sort_values(by="P-Values").iloc[:n,]


In [5]:
# Estamos assumindo que: 
# 1) Conseguimos comprar/vender o portfolio no dia em que há o cruzamento entre o z_score e o threshold. 
# 2) Não há stop loss
# 3) Não há stop de dias
# 4) As operções em aberto no final do período de teste são encerradas.

def calculate_profit(spread,threshold):
    
    #cirando variaveis
    log_ret = spread.diff() # log return eh o incremento
    dias = spread.index
    z_score = (spread-spread.mean())/spread.std()
    portfolio_return = []
    pos = 0 # 0: sem posição aberta
            # 1: Comprei o meu portfolio h = (1,-gamma)
            # -1: Vendi o meu portfolio h = -(1,-gamma)

    for i in range(1,len(z_score)):
                        
        if (z_score[i] > threshold) and (pos == 0):
            pos = -1
            print("Vendi o Portfólio no dia ",dias[i])
            #print("Preço da ação A: ",S1[i].round(2), ", Preço da ação B: ",S2[i].round(2))

        elif (z_score[i] < - threshold)  and (pos == 0):
            pos = 1            
            print("Comprei o Portfólio no dia ",dias[i])
            #print("Preço da ação A: ",S1[i].round(2), ", Preço da ação B: ",S2[i].round(2))

        else:
          if (pos == 1) and (z_score[i] < -0.75):
            portfolio_return.append(log_ret[i]*pos) # estou pegando o valor do incremento
            #print("O incremento é de: ",log_ret[i].round(2)," no dia ",dias[i])
            #print("Estou comprado, esperando o z_score = -0.75")

          elif (pos == 1) and (z_score[i] >= -0.75):
            portfolio_return.append(log_ret[i]*pos)
            print("Estava comprado, finalizei agora!",dias[i])
            #print("Preço da ação A: ",S1[i].round(2), ", Preço da ação B: ",S2[i].round(2))
            pos = 0
            

          elif (pos == -1) and (z_score[i] > 0.75):
            portfolio_return.append(log_ret[i]*pos) # estou pegando o valor do incremento
            #print("O incremento é de: ",log_ret[i].round(2)," no dia ",dias[i])
            #print("Estou vendido, esperando o z_score = 0.75")

          elif (pos == -1) and (z_score[i] <= 0.75):
            portfolio_return.append(log_ret[i]*pos)
            print("Estava vendido, finalizei agora!",dias[i])
            #print("Preço da ação A: ",S1[i].round(2), ", Preço da ação B: ",S2[i].round(2))
            pos =0
          else: 
            pos = 0


    total_ret = pd.Series(portfolio_return).sum()

    return total_ret

## Aplicação 1

### Pegando dados

In [6]:
# Getting data
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2022, 1, 1)

tickers = ['AAPL', 'ADBE', 'ORCL', 'EBAY', 'MSFT', 'QCOM', 'HPQ', 'JNPR', 'AMD', 'IBM', 'SPY']

data = pdr.get_data_yahoo(tickers, start, end)['Close']
data = np.log(data)

[*********************100%***********************]  11 of 11 completed


### Encontrando os pares Cointegrados

#### Vamos pegar os primeiros 252 dias úteis (período de formação) e verificar quais são os pares cointegrados nesse período

In [7]:
pvalues, gammas = find_cointegrated_pairs_mod(data.iloc[0:252,])
print(pvalues.shape,gammas.shape)

(11, 11) (11, 11)


### Encontrando os Top 5 pares cointegrados neste período:


In [8]:
coint_pairs_df = top_coint_pairs(data,pvalues,gammas,alpha = 0.05 , n=5)
print(coint_pairs_df)

  Stock A Stock B  P-Values     Gamma
2    AAPL    ORCL  0.005959  0.917130
1    AAPL     IBM  0.017124  0.680164
0    AAPL     HPQ  0.043885  1.280746


#### Encontrando o lucro da estratégia para os próximos 126 d.u. (período de teste)

In [9]:
resultado = []
for i in range(0,coint_pairs_df.shape[0]):
    S1_name = coint_pairs_df.iloc[i,0:2][0]
    S2_name = coint_pairs_df.iloc[i,0:2][1]
    gamma_1_2 = coint_pairs_df.iloc[i,3]

    S1 = data[S1_name].iloc[252:(252+126)] # periodo de teste
    S2 = data[S2_name].iloc[252:(252+126)] # periodo de teste

    #spread
    spread = S1 - gamma_1_2*S2

    # Pegando o resultado da estratégia
    resultado.append(np.exp(calculate_profit(spread,1.65)))
    print("-------------------------------------------------")

Vendi o Portfólio no dia  2016-01-05 00:00:00
Estava vendido, finalizei agora! 2016-01-27 00:00:00
Comprei o Portfólio no dia  2016-05-02 00:00:00
Estava comprado, finalizei agora! 2016-05-25 00:00:00
Comprei o Portfólio no dia  2016-06-29 00:00:00
-------------------------------------------------
Vendi o Portfólio no dia  2016-01-22 00:00:00
Estava vendido, finalizei agora! 2016-01-27 00:00:00
Comprei o Portfólio no dia  2016-05-11 00:00:00
Estava comprado, finalizei agora! 2016-05-24 00:00:00
-------------------------------------------------
Vendi o Portfólio no dia  2016-01-20 00:00:00
Estava vendido, finalizei agora! 2016-02-18 00:00:00
Comprei o Portfólio no dia  2016-06-02 00:00:00
Estava comprado, finalizei agora! 2016-06-27 00:00:00
-------------------------------------------------


## Aplicação 2

Agora vamos fazer uma aplicação usando os 7 anos de dados. Iremos ter um período de formação de 252 dias úteis e um período de teste de 126 dias úteis. 

In [ ]:
results = []
for j in range(0,12):

  pvalues, gammas = find_cointegrated_pairs_mod(data.iloc[126*j:(252+126*j),])
  print("-----------------")
  print("Semestre ",(1+j))
  print("-----------------")
    
  try:
    coint_pairs_df = top_coint_pairs(data,pvalues,gammas,alpha = 0.05 , n=5)

  except:
    continue

  resultado = []
  for i in range(0,coint_pairs_df.shape[0]):
      S1_name = coint_pairs_df.iloc[i,0:2][0]
      S2_name = coint_pairs_df.iloc[i,0:2][1]
      print("Par: ",S1_name,S2_name)
      gamma_1_2 = coint_pairs_df.iloc[i,3]

      S1 = data[S1_name].iloc[(252+126*j):(252+126*(j+1))] # periodo de teste
      S2 = data[S2_name].iloc[(252+126*j):(252+126*(j+1))] # periodo de teste

      #spread
      spread = S1 - gamma_1_2*S2

      # Pegando o resultado da estratégia
      resultado.append(np.exp(calculate_profit(spread,1.65)))
      print("-------------------------------------------------")
      print("                                                 ")

  results.append(resultado)

In [ ]:
results # lista de retornos dos pares ao longo do tempo.

In [12]:
# média dos retornos em cada semestre
mean_results = []
for i in range(0, len(results)):
    mean_results.append(np.prod(results[i])**(1/(len(results[i]))))



In [13]:
# média dos retornos semestrais ao longo de todo o período out-od-sample.
np.prod(mean_results)**(1/len(results))

1.1109352323660324

In [14]:
# média dos retornos ao ano.
np.prod(mean_results)**(2/len(results))

1.2341770905121703